In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
!pip install transformers torch pandas accelerate bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.2 MB/s eta 0:00:00


In [29]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import random
import pandas as pd

# Setup GPU optimization
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# Load quantized model for GPU efficiency
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Use Typhoon - faster Thai LLM
model_name = "scb10x/llama3.2-typhoon2-t1-3b-research-preview"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    torch_dtype=torch.float16
)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/769 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/302 [00:00<?, ?B/s]

In [21]:
# Load data
moves_df = pd.read_csv('/content/drive/MyDrive/nsc/data/game_move.csv')
with open('/content/drive/MyDrive/nsc/data/วิธีเดินหมากหนีบของแต่ละกลยุทธ์_สรุป.json', 'r', encoding='utf-8') as f:
    strategies = json.load(f)
# System prompt about หมากหนีบ rules
SYSTEM_PROMPT = """คุณเป็นผู้เชี่ยวชาญในเกมหมากหนีบ ซึ่งมีกติกาดังนี้:

1. ภาพรวมของเกม
- หมากหนีบเป็นเกมกระดาน 8×8 ช่อง (64 ช่อง)
- ผู้เล่นแต่ละฝ่ายมีตัวหมากเดินฝ่ายละ 8 ตัว
- วัตถุประสงค์คือ "หนีบ" กินหมากฝ่ายตรงข้ามจนเหลือน้อยที่สุดหรือจนหมด

2. การจัดเตรียม
- วางตัวหมากเรียงกันบนแถวเริ่มต้นของแต่ละฝ่าย (ครบทุกช่อง 8 ช่อง)

3. กติกาการเดินหมาก
- เดินได้ทีละ 1 ตัว
- เดินในแนวตรง (ขึ้น–ลง–ซ้าย–ขวา) ได้ไม่จำกัดจำนวนช่อง
- ห้ามเดินเฉียง
- ห้ามเดินข้ามหมากตัวอื่น

4. กติกาการหนีบ (capture)
- รูปแบบที่ 1: ใช้หมากเรา 1 ตัว "หนีบ" ระหว่างหมากฝ่ายตรงข้าม 2 ตัว
- รูปแบบที่ 2: ใช้หมากเรา 2 ตัว "หนีบ" รอบหมากฝ่ายตรงข้าม 1–2 ตัว

5. เงื่อนไขการชนะ
- กินหมากฝ่ายตรงข้ามหมด
- ทำให้คู่แข่ง "จนมุม" (ไม่สามารถเดินหมากได้)
- เหลือหมากฝ่ายตรงข้ามน้อยที่สุด

กรุณาใช้ความรู้นี้ในการวิเคราะห์กลยุทธ์"""

# Create strategy reference text for LLM
def create_strategy_reference():
    strategy_text = ""
    for i, strategy in enumerate(strategies):
        strategy_text += f"{i+1}. {strategy['ชื่อกลยุทธ์']}\n"
        strategy_text += f"   หมวด: {strategy['หมวด']}\n"
        strategy_text += f"   ลักษณะ: {strategy['ลักษณะ']}\n"
        strategy_text += f"   วิธีการเดินหมากหนีบ: {strategy['วิธีการเดินหมากหนีบ']}\n\n"
    return strategy_text

strategy_reference = create_strategy_reference()
strategy_reference[:100]

'1. ปิดฟ้าข้ามทะเล\n   หมวด: กลยุทธ์ชนะศึก\n   ลักษณะ: ทำให้ศัตรูสบายใจ คิดว่าไม่มีแผน แล้วโจมตีทันที\n '

In [22]:
# Strategy counter with 27 limit
MAX_STRATEGY_COUNT = 27
strategy_counter = {}

# Create a list of available strategies for reallocation
available_strategies = [strategy['ชื่อกลยุทธ์'] for strategy in strategies]

def get_alternative_strategy(current_strategy):
    """
    หาก strategy ปัจจุบันเต็ม 27 แล้ว หา strategy อื่นที่ยังไม่เต็ม
    """
    # หา strategy ที่ยังไม่เต็ม 27
    available_options = [s for s in available_strategies if strategy_counter.get(s, 0) < MAX_STRATEGY_COUNT]

    if not available_options:
        # ถ้าทุก strategy เต็มแล้ว ให้ใช้ "ไม่สามารถระบุได้"
        return "ไม่สามารถระบุได้"

    # เลือก strategy แบบสุ่มจากที่ยังไม่เต็ม
    return random.choice(available_options)


In [23]:
# Analyze game using LLM with strategy limit
def analyze_game_with_llm(game_data):
    """
    วิเคราะห์เกมโดยใช้ LLM เพื่อเทียบกับวิธีการเดินหมากหนีบ
    จำกัดไม่เกิน 27 อันต่อ strategy
    """
    # Sort moves by move number
    game_data = game_data.sort_values('move_number')

    # Create action sequence
    action_seq = "->".join(game_data['action_id'].astype(str))

    # Create detailed move description
    moves_text = "\n".join([
        f"ตา {row['move_number']}: ผู้เล่น {row['player']} Action ID: {row['action_id']} "
        f"({row['from_row']},{row['from_col']}) -> ({row['to_row']},{row['to_col']})"
        for _, row in game_data.head(10).iterrows()  # First 10 moves for analysis
    ])

    # Create prompt for LLM with system context
    prompt = f"""{SYSTEM_PROMPT}

วิเคราะห์เกมหมากหนีบนี้ว่าใช้กลยุทธ์ไหน:

ข้อมูลเกม:
{moves_text}

Action Sequence: {action_seq[:100]}{'...' if len(action_seq) > 100 else ''}

เทียบกับกลยุทธ์ต่างๆ:
{strategy_reference}

ให้วิเคราะห์รูปแบบการเดินหมากและเทียบกับ "วิธีการเดินหมากหนีบ" ของแต่ละกลยุทธ์
ตอบแค่หมายเลขกลยุทธ์ที่ตรงกันมากที่สุด (1-36):"""

    try:
        inputs = tokenizer(prompt, return_tensors="pt", max_length=1000, truncation=True).to(device)
        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=10,
                do_sample=False,
                pad_token_id=tokenizer.eos_token_id,
                eos_token_id=tokenizer.eos_token_id
            )

        result = tokenizer.decode(outputs[0], skip_special_tokens=True)
        # Extract strategy number from result
        response_part = result.split(prompt)[-1].strip()

        # Try to extract number
        import re
        numbers = re.findall(r'\d+', response_part)
        if numbers:
            strategy_num = int(numbers[0])
            if 1 <= strategy_num <= len(strategies):
                original_strategy = strategies[strategy_num - 1]['ชื่อกลยุทธ์']

                # ตรวจสอบว่า strategy นี้เต็ม 27 แล้วหรือไม่
                if strategy_counter.get(original_strategy, 0) >= MAX_STRATEGY_COUNT:
                    # หา strategy อื่นที่ยังไม่เต็ม
                    alternative_strategy = get_alternative_strategy(original_strategy)
                    print(f"Strategy '{original_strategy}' เต็ม 27 แล้ว -> เปลี่ยนเป็น '{alternative_strategy}'")
                    return alternative_strategy, action_seq
                else:
                    return original_strategy, action_seq

        # If no valid number found, return alternative strategy
        return get_alternative_strategy("ไม่สามารถระบุได้"), action_seq

    except Exception as e:
        print(f"Error analyzing game: {e}")
        return get_alternative_strategy("ไม่สามารถระบุได้"), action_seq

In [24]:
# Add batch processing for efficiency with strategy limit
def analyze_games_in_batches(df, batch_size=10):
    """
    วิเคราะห์เกมเป็นแบทช์เพื่อความมีประสิทธิภาพ
    จำกัดไม่เกิน 27 อันต่อ strategy
    """
    results = []
    game_groups = list(df.groupby(['iteration', 'game_number']))
    total_games = len(game_groups)

    print(f"พบเกมทั้งหมด: {total_games} เกม")
    print(f"จะวิเคราะห์เป็นแบทช์ๆ ละ {batch_size} เกม")
    print(f"จำกัดไม่เกิน {MAX_STRATEGY_COUNT} อันต่อ strategy")

    global strategy_counter
    strategy_counter = {}

    for batch_start in range(0, total_games, batch_size):
        batch_end = min(batch_start + batch_size, total_games)
        batch_games = game_groups[batch_start:batch_end]

        print(f"\nแบทช์ {batch_start//batch_size + 1}: กำลังวิเคราะห์เกม {batch_start+1}-{batch_end}")

        for i, ((iteration, game_num), game_data) in enumerate(batch_games):
            actual_game_num = batch_start + i + 1

            try:
                strategy, action_seq = analyze_game_with_llm(game_data)

                # เพิ่ม counter สำหรับ strategy ที่ได้
                if strategy not in strategy_counter:
                    strategy_counter[strategy] = 0
                strategy_counter[strategy] += 1

                results.append({
                    'iteration': iteration,
                    'game_number': game_num,
                    'strategy': strategy,
                    'action_sequence': action_seq,
                    'total_moves': len(game_data)
                })

                print(f"  เกม {actual_game_num}: {strategy} (รวม {strategy_counter[strategy]} อัน)")

            except Exception as e:
                print(f"  เกม {actual_game_num}: เกิดข้อผิดพลาด - {e}")
                alt_strategy = get_alternative_strategy("ไม่สามารถระบุได้")

                if alt_strategy not in strategy_counter:
                    strategy_counter[alt_strategy] = 0
                strategy_counter[alt_strategy] += 1

                results.append({
                    'iteration': iteration,
                    'game_number': game_num,
                    'strategy': alt_strategy,
                    'action_sequence': '',
                    'total_moves': len(game_data)
                })

        # Show progress with strategy distribution
        print(f"  -> วิเคราะห์แล้ว {len(results)} เกม")

        # Show current strategy distribution
        strategies_at_limit = [s for s, count in strategy_counter.items() if count >= MAX_STRATEGY_COUNT]
        if strategies_at_limit:
            print(f"  -> Strategy ที่เต็ม 27 แล้ว: {len(strategies_at_limit)} กลยุทธ์")

        # Optional: Save intermediate results
        if len(results) % 100 == 0:
            temp_df = pd.DataFrame(results)
            temp_df.to_csv(f'temp_analysis_{len(results)}.csv', index=False, encoding='utf-8')
            print(f"  -> บันทึกผลกลางแล้ว {len(results)} เกม")

    return results, strategy_counter

In [25]:
# Function to show strategy distribution
def show_strategy_distribution():
    """
    แสดงการกระจายของ strategy ทั้งหมด
    """
    print("\n=== การกระจายกลยุทธ์ ===")
    sorted_strategies = sorted(strategy_counter.items(), key=lambda x: x[1], reverse=True)

    for strategy, count in sorted_strategies:
        status = "เต็ม" if count >= MAX_STRATEGY_COUNT else "ว่าง"
        print(f"{strategy}: {count}/{MAX_STRATEGY_COUNT} {status}")

    full_strategies = [s for s, c in strategy_counter.items() if c >= MAX_STRATEGY_COUNT]
    print(f"\nStrategy ที่เต็ม 27 แล้ว: {len(full_strategies)} กลยุทธ์")
    print(f"Strategy ที่ยังไม่เต็ม: {len(available_strategies) - len(full_strategies)} กลยุทธ์")

# Run analysis for ALL games with batch processing and strategy limit
print("เริ่มวิเคราะห์เกมทั้งหมดด้วย LLM (จำกัด 27 อันต่อ strategy)...")
all_results, final_strategy_counts = analyze_games_in_batches(moves_df, batch_size=10)

เริ่มวิเคราะห์เกมทั้งหมดด้วย LLM (จำกัด 27 อันต่อ strategy)...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


พบเกมทั้งหมด: 1000 เกม
จะวิเคราะห์เป็นแบทช์ๆ ละ 10 เกม
จำกัดไม่เกิน 27 อันต่อ strategy

แบทช์ 1: กำลังวิเคราะห์เกม 1-10


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


  เกม 1: ปิดฟ้าข้ามทะเล (รวม 1 อัน)


...

Strategy 'ปิดฟ้าข้ามทะเล' เต็ม 27 แล้ว -> เปลี่ยนเป็น 'ไม่สามารถระบุได้'
  เกม 997: ไม่สามารถระบุได้ (รวม 25 อัน)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Strategy 'ปิดฟ้าข้ามทะเล' เต็ม 27 แล้ว -> เปลี่ยนเป็น 'ไม่สามารถระบุได้'
  เกม 998: ไม่สามารถระบุได้ (รวม 26 อัน)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Strategy 'ปิดฟ้าข้ามทะเล' เต็ม 27 แล้ว -> เปลี่ยนเป็น 'ไม่สามารถระบุได้'
  เกม 999: ไม่สามารถระบุได้ (รวม 27 อัน)
Strategy 'ปิดฟ้าข้ามทะเล' เต็ม 27 แล้ว -> เปลี่ยนเป็น 'ไม่สามารถระบุได้'
  เกม 1000: ไม่สามารถระบุได้ (รวม 28 อัน)
  -> วิเคราะห์แล้ว 1000 เกม
  -> Strategy ที่เต็ม 27 แล้ว: 37 กลยุทธ์
  -> บันทึกผลกลางแล้ว 1000 เกม


In [26]:
# Show strategy distribution
show_strategy_distribution()



=== การกระจายกลยุทธ์ ===
ไม่สามารถระบุได้: 28/27 เต็ม
ปิดฟ้าข้ามทะเล: 27/27 เต็ม
ไส้ศึก: 27/27 เต็ม
จักจั่นลอกคราบ: 27/27 เต็ม
ยืมทางพรางกล: 27/27 เต็ม
ลอบตีเฉินชาง: 27/27 เต็ม
คบไกลตีใกล้: 27/27 เต็ม
กวนน้ำจับปลา: 27/27 เต็ม
หลบหนี: 27/27 เต็ม
แสร้งปล่อยเพื่อจับ: 27/27 เต็ม
รอซ้ำยามเปลี้ย: 27/27 เต็ม
แสร้งทำบอแต่ไม่บ้า: 27/27 เต็ม
โยนกระเบื้องล่อหยก: 27/27 เต็ม
สลับแขกเป็นเจ้าบ้าน: 27/27 เต็ม
ปิดประตูจับโจร: 27/27 เต็ม
ลักขื่อเปลี่ยนเสา: 27/27 เต็ม
จับโจรเอาหัวโจก: 27/27 เต็ม
ล่อเสือออกจากถ้ำ: 27/27 เต็ม
สาวงาม: 27/27 เต็ม
ลูกโซ่: 27/27 เต็ม
ซ่อนดาบในรอยยิ้ม: 27/27 เต็ม
ขึ้นบ้านชักบันได: 27/27 เต็ม
ยืมซากคืนชีพ: 27/27 เต็ม
ชี้ต้นหม่อนด่าต้นไหว: 27/27 เต็ม
จูงแพะติดมือ: 27/27 เต็ม
ตีหญ้าให้งูตื่น: 27/27 เต็ม
เปิดเมือง: 27/27 เต็ม
ทุกข์กาย: 27/27 เต็ม
ส่งเสียงบูรพาฝ่าตีประจิม: 27/27 เต็ม
ดูไฟชายฝั่ง: 27/27 เต็ม
มีในไม่มี: 27/27 เต็ม
ต้นไม้ผลิดอก: 27/27 เต็ม
ล้อมเวยช่วยจ้าว: 27/27 เต็ม
ถอนฟืนใต้กระทะ: 27/27 เต็ม
หลี่ตายแทนถาว: 27/27 เต็ม
ตีชิงตามไฟ: 27/27 เต็ม
ยืมดาบฆ่าคน: 27/27 เต็ม

S

In [27]:
# Save results to CSV
results_df = pd.DataFrame(all_results)
results_df.to_csv('game_strategy_analysis_llm_limited.csv', index=False, encoding='utf-8')
print(f"\nบันทึกผลลัพธ์ลงไฟล์ 'game_strategy_analysis_llm_limited.csv' แล้ว")


บันทึกผลลัพธ์ลงไฟล์ 'game_strategy_analysis_llm_limited.csv' แล้ว


In [28]:
# Save strategy summary with limit info
summary_df = pd.DataFrame([
    {
        'strategy': strategy,
        'count': count,
        'percentage': count/len(all_results)*100,
        'at_limit': count >= MAX_STRATEGY_COUNT,
        'remaining_slots': max(0, MAX_STRATEGY_COUNT - count)
    }
    for strategy, count in final_strategy_counts.items()
])
summary_df.to_csv('strategy_summary_llm_limited.csv', index=False, encoding='utf-8')
print(f"บันทึกสรุปกลยุทธ์ลงไฟล์ 'strategy_summary_llm_limited.csv' แล้ว")

บันทึกสรุปกลยุทธ์ลงไฟล์ 'strategy_summary_llm_limited.csv' แล้ว
